In [8]:
!ls ../../evaluations/hate_speech/task_b/

beto-hierarchical-gamma-0.1.json  beto.json


In [7]:
evaluations["beto-hier"]

{'model': 'dccuchile/bert-base-spanish-wwm-uncased',
 'lang': 'es',
 'train_args': {'epochs': 5,
  'batch_size': 32,
  'eval_batch_size': 16,
  'warmup_ratio': 0.1,
  'limit': None,
  'task_b': True,
  'metric_for_best_model': 'emr',
  'hierarchical': True,
  'gamma': 0.005},
 'evaluations': {'hate_speech': [{'eval_loss': 22.860462188720703,
    'eval_hs_f1': 0.7242105263157895,
    'eval_hs_precision': 0.6745098039215687,
    'eval_hs_recall': 0.7818181818181819,
    'eval_tr_f1': 0.7445255474452555,
    'eval_tr_precision': 0.7669172932330827,
    'eval_tr_recall': 0.723404255319149,
    'eval_ag_f1': 0.6778711484593837,
    'eval_ag_precision': 0.6080402010050251,
    'eval_ag_recall': 0.7658227848101266,
    'eval_macro_hs_f1_score': 0.7514010378057822,
    'eval_emr_no_gating': 0.6275,
    'eval_emr': 0.66625,
    'eval_macro_f1': 0.7155357003211975,
    'eval_macro_precision': 0.6831557750701904,
    'eval_macro_recall': 0.75701504945755,
    'eval_runtime': 3.1087,
    'eval_sam

In [37]:
import json
import pandas as pd

evaluations = {

}

for model_name, path in [
    ('beto', '../../evaluations/hate_speech/task_b/beto.json'),
    ('beto-hier', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.1.json'),
    #('robertuito', '../../evaluations/hate_speech/robertuito-taskb.json'),
    #('robertuito-hier', '../../evaluations/hate_speech/robertuito-taskb-hier.json'),
    #('robertuito-deacc', '../../evaluations/robertuito-deacc-hate-speech-task-b.json'),
    ]:
    with open(path) as f:
        evaluations[model_name] = json.load(f)


for key, evals in evaluations.items():
    print(f"We have {len(evals['evaluations']['hate_speech'])} {key} evaluations")



dfs = []
for model_name, model_results in evaluations.items():
    model_evaluations = model_results["evaluations"]["hate_speech"]
    
    if not model_evaluations:
        continue
    
    df = pd.DataFrame(model_evaluations)

    df.columns = [x.split("_", 1)[1] if "_" in x else x for x in df.columns]
    
    mean_df = pd.DataFrame({
        f"{model_name} mean": df.mean(), 
        f"{model_name} std": df.std()
    })
    dfs.append(mean_df)

result_df = pd.concat(dfs, axis=1)
index = [
    'hs_f1', 'hs_precision', 'hs_recall', 'tr_f1',
    'ag_f1', 'macro_hs_f1_score',
    'emr_no_gating', 'emr', 
    'macro_f1',
]
result_df.loc[index]

We have 10 beto evaluations
We have 10 beto-hier evaluations


,beto mean,beto std,beto-hier mean,beto-hier std
hs_f1,0.741171,0.013294,0.734925,0.013436
hs_precision,0.712582,0.041683,0.743658,0.020801
hs_recall,0.777879,0.054317,0.727576,0.030116
tr_f1,0.765226,0.012095,0.758053,0.014196
ag_f1,0.687683,0.014856,0.674417,0.013812
macro_hs_f1_score,0.771392,0.014996,0.776030,0.010041
emr_no_gating,0.684063,0.025306,0.673875,0.016543
emr,0.684812,0.025478,0.703313,0.011592
macro_f1,0.731360,0.011938,0.722465,0.009771


In [43]:
import scipy.stats

beto_hier_emrs = [evaluation["eval_emr"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]]
beto_emrs = [evaluation["eval_emr"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]]

scipy.stats.mannwhitneyu(beto_hier_emrs, beto_emrs, alternative="greater")

MannwhitneyuResult(statistic=73.5, pvalue=0.04093431186532218)